In [1]:
import spacy
import pandas as pd

In [2]:
nlp = spacy.load('fr')

In [3]:
from lxml import etree
from datetime import timedelta, date


In [4]:
def month_dates(start, end):
    f = lambda date: date.month + 12 * date.year

    res = []
    for tot_m in range(f(start)-1, f(end)):
        y, m = divmod(tot_m, 12)
        res.append(str(y) + '/' + '%02d' % (m+1))
    
    return res

In [5]:
# retrieve the articles from JDL dataset
start = date(1826, 1, 1); end = date(1998, 2, 28)
articles = []
for date in month_dates(start, end):
    path = '/home/mbanga/Desktop/JDG/' + date + '.xml'
    try:
        parser = etree.parse(path)
        tag_text = 'full_text'

        for article in parser.getroot().iterchildren():
            a = ''
            for elem in article.iterchildren():
                a += elem.findtext(tag_text) + ' '
            articles.append(a)

    except (FileNotFoundError, IOError):
        pass

In [12]:
len(articles)

3080369

In [25]:
test_corpus = articles[-50:]

In [30]:
parsed_article = nlp(test_corpus[48])

In [31]:
for num, sentence in enumerate(parsed_article.sents):
    print('Sentence {}:' .format(num + 1))
    print(sentence)
    print('')

Sentence 1:
Mœurs Pamela Anderson demande (encore) le divorce Pamela Anderson a demandé jeudi à divorcer de son mari, le batteur Tommy Lee. Ce dernier a été arrêté deux jours plus tôt à leur domicile pour avoir agressé l'actrice. Il s'agit de ta deuxième requête en divorce déposée par Pamela Anderson en deux ans. Le couple s'était réconcilié après la naissance de leur premier fils, Brandon, et Pamela Anderson avait dit qu'elle voulait aider son époux à vaincre son alcoolisme. Selon l'actrice, Tommy Lee l'a attaquée alors qu'elle ber-çait leur fils de sept semaines, Dylan Jagger. Elle a été légèrement blessée à la main et présente des marques rouges sur le dos. La police appelée à leur domicile de Malibu y a découvert une arme, en violation du sursis avec mise à l'épreuve auquel le batteur était soumis pour avoir agressé un photographe. (ATS / Reuter) Vin Attention aux mélanges... Le Tribunal de Sierre a confirmé vendredi la peine de trois mois de prison avec sursis contre un encaveur q

In [40]:
token_pos = [token.pos_ for token in parsed_article]
token_text = [token.orth_ for token in parsed_article]
token_shape = [token.shape_ for token in parsed_article]
token_oov = [token.is_oov for token in parsed_article]

t = pd.Series(token_text, name='token_text')
p = pd.Series(token_pos, name='token_pos')
s = pd.Series(token_shape, name='token_shape')
oov = pd.Series(token_oov, name='token_oov')

df = pd.concat([t, p, s, oov], axis=1)

df.head()

,token_text,token_pos,token_shape,token_oov
0,Mœurs,PROPN,Xxxxx,True
1,Pamela,PROPN,Xxxxx,True
2,Anderson,PROPN,Xxxxx,True
3,demande,VERB,xxxx,True
4,(,PUNCT,(,True


In [33]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

In [34]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuaiton or whitespace
    """
    
    return token.is_punct or token.is_space

In [44]:
def lemmatized_sentence_corpus(corpus):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    j = 0
    i = 0
    for parsed_article in nlp.pipe(corpus, 
                                   batch_size=50, n_threads=1):
        
        yield u' '.join([token.lemma_ for token in parsed_article
                             if not punct_space(token)])

In [45]:
for lemma in lemmatized_sentence_corpus(test_corpus):
    print(lemma)
    print('\n')

ils ont osé le peuple suisse tout entier est bouleversé la consternation se lit sur tous les visages nous avions mal ces jours en songeant à l' implacable déferlement des colonnes blindées soviétiques sur les routes de hongrie un faible espoir pourtant subsistait ils n' oseront pas ils ont osé staline a trouvé pour lui succéder dans le crime des émules qui le valent en cynisme et en brutalité a quoi bon accumuler les mots nous sommes tous au clair sur nos sentiments il n' est pas un suisse digne de ce nom qui ne puisse par exemple souscrire à la réponse que l' union syndicale vient de communiquer à l' ambassade sovié tique à berne que pouvons nous que devonsnous faire un vaste mouvement se dessine dans le pays en faveur d' une rupture des relations diplomatiques avec moscou là n' est pas la voie dans laquelle il nous faut nous engager l' existence de l' etat soviétique est un journal de genève du 5 novembre 1956 fait nous ne pouvons l' ignorer avec cet etat nous avons nécessairement de

In [41]:
LineSentence(test_corpus[48])